In [ ]:
from ogb.nodeproppred import NodePropPredDataset
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
dataset = NodePropPredDataset(name='ogbn-arxiv', root='./arxiv/')

In [ ]:
graph, label = dataset[0] # graph: library-agnostic graph object

In [ ]:
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

In [ ]:
nodelabels = pd.Series(label.flatten(), name="label")

In [ ]:
nodeid2paperid = pd.read_csv("arxiv/ogbn_arxiv/mapping/nodeidx2paperid.csv", dtype=str)
nodeid2paperid

In [ ]:
nodeid2paperid2label = pd.merge(nodeid2paperid, nodelabels, left_index=True, right_index=True)
nodeid2paperid2label

In [ ]:
paperid2text = pd.read_csv("arxiv/titleabs.tsv", sep="\t", dtype=str, names=["paper id", "title", "abstract"])
paperid2text

In [ ]:
nodeid2text = pd.merge(nodeid2paperid2label, paperid2text, on="paper id")
nodeid2text = nodeid2text.assign(text=nodeid2text["title"].str.cat(nodeid2text["abstract"], sep=" "))
nodeid2text = nodeid2text[["node idx", "label", "text"]]
nodeid2text

In [ ]:
nodeid2text_train = nodeid2text.loc[train_idx]
nodeid2text_valid = nodeid2text.loc[valid_idx]
nodeid2text_test = nodeid2text.loc[test_idx]

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(nodeid2text_train["text"])
X_train_counts = vectorizer.transform(nodeid2text_train["text"])

In [ ]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train = tf_transformer.transform(X_train_counts)
X_train

In [ ]:
clf = MultinomialNB().fit(X_train, nodeid2text_train["label"])

In [ ]:
valid_pred = clf.predict(tf_transformer.transform(vectorizer.transform(nodeid2text_valid["text"])))
test_pred = clf.predict(tf_transformer.transform(vectorizer.transform(nodeid2text_test["text"])))

In [ ]:
(valid_pred == nodeid2text_valid["label"]).mean()

In [ ]:
(test_pred == nodeid2text_test["label"]).mean()

In [ ]:
len(nodeid2text["label"].unique())